In [2]:
import numpy as np
import json
from pathlib import Path
import shutil

In [27]:
base_dir = Path("./")
for scenario_dir in base_dir.iterdir():
    if not scenario_dir.is_dir():
        continue

    for data_file in scenario_dir.glob("onlyq_*.json"):
        
        # copy data_file 
        data_file_backup = data_file.with_suffix(".json.bak")
        shutil.copy(data_file, data_file_backup)
        with open(data_file, "r", encoding="utf-8") as f:
            datas = json.loads(f.read())

        with open(data_file, "w", encoding="utf-8") as f:
            new_datas = []
            for data in datas:
                input, response = data["Input"], data["Response"]
                formalized_input, input_semantic_parsing, strategy = \
                    response["Formalized Input"], response["Input Semantic Parsing"], response["Strategy"]
                
                isp_flatten = {}
                for k, v in input_semantic_parsing.items():
                    isp_flatten.update(v)
                
                isp_flatten = [f"'{k}':'{v}'" for k, v in isp_flatten.items()]

                steps = strategy["Step"]
                steps = '\n'.join([f"{i+1}. {step}" for i, step in enumerate(steps)])

                cot_text = \
f"""
사용자는 '{formalized_input}'라고 입력함. {', '.join(isp_flatten)}에 해당함.
사용자의 의도를 추측하면, 유저는 '{', '.join(strategy['Expected Outputs'])}' 와 비슷한 답변을 기대할것.
퀄리티 있는 대답을 위해 나는 다음 단계를 수행.
{steps}
"""
                # print(cot_text)

                new_response = {
                    '생각': cot_text,
                    '지시': response['Instruction Set'],
                }

                new_data = {
                    'Input': input,
                    'Response': new_response,
                }
                new_datas.append(new_data)
            f.write(json.dumps(new_datas, ensure_ascii=False, indent=4))